In [1]:
import os
from glob import glob
import pandas as pd
from functools import reduce
from xml.etree import ElementTree as et

In [2]:
# load all xml files and store in list
xml_list = glob('./data_images/*.xml')
xml_list

['./data_images/007826.xml',
 './data_images/002786.xml',
 './data_images/006286.xml',
 './data_images/002962.xml',
 './data_images/008297.xml',
 './data_images/009189.xml',
 './data_images/009823.xml',
 './data_images/002976.xml',
 './data_images/002745.xml',
 './data_images/006523.xml',
 './data_images/008268.xml',
 './data_images/004452.xml',
 './data_images/002023.xml',
 './data_images/005980.xml',
 './data_images/004446.xml',
 './data_images/002037.xml',
 './data_images/009162.xml',
 './data_images/006251.xml',
 './data_images/000620.xml',
 './data_images/000146.xml',
 './data_images/007629.xml',
 './data_images/001258.xml',
 './data_images/002751.xml',
 './data_images/002989.xml',
 './data_images/007601.xml',
 './data_images/001270.xml',
 './data_images/002779.xml',
 './data_images/005016.xml',
 './data_images/003301.xml',
 './data_images/006279.xml',
 './data_images/007167.xml',
 './data_images/008254.xml',
 './data_images/000608.xml',
 './data_images/005764.xml',
 './data_image

In [21]:
def extract_text(filename):
    tree = et.parse(filename)
    root = tree.getroot()
    image_name = root.find('filename').text
    width = float(root.find('size').find('width').text)
    height = float(root.find('size').find('height').text)
    objs = root.findall('object')
    parser = []
    for obj in objs:
        name = obj.find('name').text
        xmin =  obj.find('bndbox').find('xmin').text
        xmax = obj.find('bndbox').find('xmax').text
        ymin =  obj.find('bndbox').find('xmin').text
        ymax = obj.find('bndbox').find('xmin').text

        parser.append([image_name,width,height,name, \
                       xmin,xmax,ymin,ymax])
    return parser






In [23]:
parser_all = list(map(extract_text,xml_list))
data = reduce(lambda x,y : x+y, parser_all)
df = pd.DataFrame(data,columns = ['filename','width','height','name','xmin','xmax','ymin','ymax'])
cols = ['width','height','xmin','xmax','ymin','ymax']
df[cols] = df[cols].astype(int)
df['center_x'] = ((df['xmax']+df['xmin'])/2)/df['width']
df['center_y'] = ((df['ymax']+df['ymin'])/2)/df['height']
df['w'] = ((df['xmax'] - df['xmin'])/df['width'])
df['h'] = ((df['ymax'] - df['ymin']/df['height']))
df.head()
#df['name'].value_counts()

,filename,width,height,name,xmin,xmax,ymin,ymax,center_x,center_y,w,h
0,./data_images/007826.xml,500,375,diningtable,80,320,80,80,0.400,0.213333,0.480,79.786667
1,./data_images/007826.xml,500,375,chair,197,257,197,197,0.454,0.525333,0.120,196.474667
2,./data_images/007826.xml,500,375,chair,139,185,139,139,0.324,0.370667,0.092,138.629333
3,./data_images/007826.xml,500,375,chair,258,312,258,258,0.570,0.688000,0.108,257.312000
4,./data_images/007826.xml,500,375,chair,10,93,10,10,0.103,0.026667,0.166,9.973333


Split Data into train and Test sets

In [36]:
#split data into train and test data sets
images = df['filename'].unique()
img_df = pd.DataFrame(images,columns=['filename'])
img_df.head()
img_train = tuple(img_df.sample(frac=0.8)['filename']) #shuffle and pick 80% of images
img_test = tuple(img_df.query(f'filename not in {img_train}')['filename'])

train_df = df.query(f'filename in {img_train}')
test_df = df.query(f'filename in {img_test}')
len(img_train),len(img_test)
len(train_df),len(test_df)
#test_df.head()


(12578, 3085)

In [37]:
train_df.head()

,filename,width,height,name,xmin,xmax,ymin,ymax,center_x,center_y,w,h
0,./data_images/007826.xml,500,375,diningtable,80,320,80,80,0.400,0.213333,0.480,79.786667
1,./data_images/007826.xml,500,375,chair,197,257,197,197,0.454,0.525333,0.120,196.474667
2,./data_images/007826.xml,500,375,chair,139,185,139,139,0.324,0.370667,0.092,138.629333
3,./data_images/007826.xml,500,375,chair,258,312,258,258,0.570,0.688000,0.108,257.312000
4,./data_images/007826.xml,500,375,chair,10,93,10,10,0.103,0.026667,0.166,9.973333


In [38]:
test_df.head()

,filename,width,height,name,xmin,xmax,ymin,ymax,center_x,center_y,w,h
32,./data_images/008297.xml,500,375,bird,2,500,2,2,0.502,0.005333,0.996,1.994667
49,./data_images/004452.xml,500,375,aeroplane,411,454,411,411,0.865,1.096000,0.086,409.904000
56,./data_images/006251.xml,500,375,sheep,75,146,75,75,0.221,0.200000,0.142,74.800000
57,./data_images/006251.xml,500,375,sheep,160,201,160,160,0.361,0.426667,0.082,159.573333
58,./data_images/006251.xml,500,375,sheep,127,179,127,127,0.306,0.338667,0.104,126.661333


Label Encoding

In [46]:
#label encoding
def label_encoding(x):
    labels = {'person':0,'car':1,'chair':2,'bottle':3,'pottedplant':4,'bird':5,'dog':6,\
              'sofa':7,'bicycle':8,'horse':9,'boat':10,'motorbike':11,'cat':12,'tvmonitor':13,\
                'cow':14,'sheep':15,'aeroplane':16,'train':17,'diningtable':18,'bus':19}
    return labels[x]

In [76]:
label_encoding(train_df['name'][3])

train_df['id'] = train_df['name'].apply(label_encoding)
test_df['id'] = test_df['name'].apply(label_encoding)

train_df.head()

/var/folders/dn/xcm39yl564qcdvnnd_msgkcw0000gn/T/ipykernel_87525/676248809.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['id'] = train_df['name'].apply(label_encoding)
/var/folders/dn/xcm39yl564qcdvnnd_msgkcw0000gn/T/ipykernel_87525/676248809.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['id'] = test_df['name'].apply(label_encoding)


,filename,width,height,name,xmin,xmax,ymin,ymax,center_x,center_y,w,h,id
0,./data_images/007826.xml,500,375,diningtable,80,320,80,80,0.400,0.213333,0.480,79.786667,18
1,./data_images/007826.xml,500,375,chair,197,257,197,197,0.454,0.525333,0.120,196.474667,2
2,./data_images/007826.xml,500,375,chair,139,185,139,139,0.324,0.370667,0.092,138.629333,2
3,./data_images/007826.xml,500,375,chair,258,312,258,258,0.570,0.688000,0.108,257.312000,2
4,./data_images/007826.xml,500,375,chair,10,93,10,10,0.103,0.026667,0.166,9.973333,2


Save image and labels in train and test folders

In [78]:
import os
from shutil import move

train_folder = 'data_images/train'
test_folder = 'data_images/test'

os.mkdir(train_folder)
os.mkdir(test_folder)


In [89]:
cols = ['filename','id','center_x','center_y','w','h']
groupby_obj_train = train_df[cols].groupby('filename')
groupby_obj_test = test_df[cols].groupby('filename')

groupby_obj_train.head()

,filename,id,center_x,center_y,w,h
0,./data_images/007826.xml,18,0.400000,0.213333,0.480000,79.786667
1,./data_images/007826.xml,2,0.454000,0.525333,0.120000,196.474667
2,./data_images/007826.xml,2,0.324000,0.370667,0.092000,138.629333
3,./data_images/007826.xml,2,0.570000,0.688000,0.108000,257.312000
4,./data_images/007826.xml,2,0.103000,0.026667,0.166000,9.973333
...,...,...,...,...,...,...
15658,./data_images/004488.xml,1,0.692000,0.864000,0.088000,323.136000
15659,./data_images/005796.xml,0,0.452000,0.090000,0.664000,44.910000
15660,./data_images/005796.xml,13,0.900000,0.600000,0.200000,299.400000
15661,./data_images/002947.xml,7,0.508000,0.012000,0.984000,5.988000
